In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict

In [2]:
START_YEAR = 1996
END_YEAR = 2018
dfs = {}

for year in range(START_YEAR, END_YEAR):
    year_YY = year % 100
    dfs[year] = pd.read_csv('CollegeScorecard_Raw_Data/MERGED' + \
                            str(year) + '_{:02d}_PP.CSV'.format((year_YY + 1) % 100), delimiter=',')

//anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
//anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (1806,1814) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
//anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (1,6,442,451,598,607,611,620,624,633,789,802,806,815,971,984,997,1153,1166,1407,1408,1411,1424,1425,1426,1431,1432,1433,1437,1438,1439,1440,1445,1446,1447,1451,1452,1453,1454,1459,1460,1461,1465,1466,1467,1468,1473,1474,1475,1479,1480,1481,1482,1487,1488,1489,1501,1502,1537,1538,1539,1540,1541,1542,1606,1609,1610,1611,1613,1614,1615,1616,1688,1689,1690,1691,1692,1908,1909,1910,1911,1912,1913) have mi

In [3]:
NULL_THRESHOLD = 0.5
columns_to_drop = set()
t=0
useful_labs = defaultdict(int)
labels = ['MD_EARN_WNE_P6', 'MD_EARN_WNE_P8', 'MD_EARN_WNE_P10', \
          'MN_EARN_WNE_P6', 'MN_EARN_WNE_P8', 'MN_EARN_WNE_P10', \
          'CDR2_DENOM', 'CDR3_DENOM']

for lab in labels:
    for year in dfs:
        df = dfs[year]

        d= df[lab]
        if d.count() > d.isna().sum():
            useful_labs[lab] += 1
            #print(lab + str(year - int(lab[-1])))

#Number of years where more than 50% of the corresponding label data is present
print(useful_labs)

defaultdict(<class 'int'>, {'MD_EARN_WNE_P6': 8, 'MD_EARN_WNE_P8': 7, 'MD_EARN_WNE_P10': 6, 'MN_EARN_WNE_P6': 8, 'MN_EARN_WNE_P8': 7, 'MN_EARN_WNE_P10': 6, 'CDR2_DENOM': 17, 'CDR3_DENOM': 7})


In [4]:
earn_start_year = 0
earn_end_year = 0
earn_year_has_data = {}

for year in dfs:
    df = dfs[year]

    d= df['MD_EARN_WNE_P6']
    if d.count() > d.isna().sum():
        if earn_start_year == 0:
            earn_start_year = year
        else:
            earn_end_year = year
        earn_year_has_data[year] = True

In [5]:
missing_years = {}
for year in range(earn_start_year, earn_end_year):
    if year not in earn_year_has_data:
        missing_years[year] = True
print(missing_years)

{2004: True, 2006: True, 2008: True, 2010: True}


In [6]:
df_year_earnings = pd.DataFrame(columns = ['UNITID', 'Year', 'MD_EARN_WNE_P6']) 

for year in range(earn_start_year, earn_end_year):
    if year not in missing_years:
        df = dfs[year]
        df['Year'] = year
        #print(df[['UNITID', 'Year', 'MD_EARN_WNE_P6']])
        df_year_earnings = df_year_earnings.append(df[['UNITID', 'Year', 'MD_EARN_WNE_P6']])
#print(df_year_earnings)
print(df_year_earnings.isna().sum())
print(df_year_earnings.isna().count())

set_ids = set(df_year_earnings['UNITID'])

UNITID               0
Year                 0
MD_EARN_WNE_P6    4055
dtype: int64
UNITID            50720
Year              50720
MD_EARN_WNE_P6    50720
dtype: int64


In [ ]:
from scipy.interpolate import InterpolatedUnivariateSpline

for unit_id in set_ids:
    entry = df_year_earnings.loc[df_year_earnings['UNITID'] == unit_id].sort_values(by=['Year'])
    
    #need to clean data and remove data with missing years
    if len(entry.index) < earn_end_year - earn_start_year - len(missing_years):
        df_year_earnings[df_year_earnings.UNITID != unit_id]
        continue
        
    x = entry['Year']
    y = entry['MD_EARN_WNE_P6']
    try:
        spl = InterpolatedUnivariateSpline(x, y)
    except ValueError:
        data = [unit_id, year, float(interp_earn)]
        new_earn_df = pd.DataFrame([{'UNITID': data[0], 'Year': data[1], 'MD_EARN_WNE_P6': np.NaN}]) 
        df_year_earnings = df_year_earnings.append(new_earn_df)
        continue
        
    for year in missing_years:
        interp_earn = spl(year)
        data = [unit_id, year, float(interp_earn)]
        new_earn_df = pd.DataFrame([{'UNITID': data[0], 'Year': data[1], 'MD_EARN_WNE_P6': data[2]}]) 
        df_year_earnings = df_year_earnings.append(new_earn_df)
    

//anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [ ]:
#open questions...
#drop all rows with NaNs/privacy suppressed for median earnings?

print(df_year_earnings[['UNITID', 'Year', 'MD_EARN_WNE_P6']])
#print(sorted(columns_to_drop))
#columns_to_drop.update(df.columns[df.isnull().mean() < NULL_THRESHOLD].values)
#        r =  dfs[year]['MD_EARN_WNE_P' + p].isna().sum() / (dfs[year]['MD_EARN_WNE_P' + p].count() + dfs[year]['MD_EARN_WNE_P6'].isna().sum() )
#print(len(columns_to_drop))
#print(len(dfs[2000].columns))

In [ ]:
print(df_year_earnings.loc[df_year_earnings['UNITID']==100690].sort_values(by=['Year']))